In [1]:
import mlagents
from mlagents_envs.environment import UnityEnvironment as UE
import numpy as np
from mlagents_envs.environment import ActionTuple


In [2]:
# env.close()

In [3]:
# env =  UE(file_name="stage0_160523\stage0_copy",seed=1,side_channels=[])
file_name = "C:\\Users\\Palaash.HPZ\\Desktop\\RL-concept-learning_large_build_envs\\build_envs\\windows\\S2 180723\\build"
env =  UE(file_name=file_name,seed=1,side_channels=[],worker_id=2,no_graphics = False)
env.reset()

In [4]:
# model
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

vision_output_dim = 3136
num_words = 35  # Number of unique words in the vocabulary
bert_output_dim = 768  # output dim of BERT
language_output_dim = 128
embedding_dim = 128
mixing_dim = 256
lstm_hidden_dim = 256
num_actions = 4

# (3,128,128) --> (64,7,7) = 3136 (3-layer CNN)
class VisualModule(nn.Module): 
    def __init__(self):
        super(VisualModule, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=3, padding=0),
            nn.ReLU()
        )
        # self.conv = nn.Sequential(
        #     nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(128, 64, kernel_size=5, stride=2, padding=1),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        #     nn.ReLU(),
        # )

    def forward(self, vt):
        encoded_vt = self.conv(vt)
        return encoded_vt.view(vt.size(0), -1).squeeze()

# one-hot encoding [0 0 1 0 0] --> 128 dimensional embedding (FF)
# S1:5 S2:5 S3:11 S4:9 --> 30 + 5 (noun) = 35 in total
class LanguageModule(nn.Module): 
    def __init__(self, num_words, embedding_dim):
        super(LanguageModule, self).__init__()
        self.embedding = nn.Linear(num_words, embedding_dim)

    def forward(self, lt):
        embedded_lt = self.embedding(lt)
        return embedded_lt

# 3136(vision) + 128 (language) --> 256 dimensional embedding (FF)
class MixingModule(nn.Module):
    def __init__(self, vision_output_dim, language_output_dim, mixing_dim):
        super(MixingModule, self).__init__()
        self.linear = nn.Linear(vision_output_dim + language_output_dim, mixing_dim)

    def forward(self, vision_output, language_output):
        combined_output = torch.cat((vision_output, language_output), dim=0)
        mixed_output = self.linear(combined_output)
        return mixed_output

class LSTMModule(nn.Module):
    def __init__(self,mixing_dim,lstm_hidden_dim):
        super(LSTMModule, self).__init__()
        self.lstm = nn.LSTMCell(mixing_dim, lstm_hidden_dim)
    
    def forward(self,mixed_output,lstm_hidden_state):
        lstm_hidden_state = self.lstm(mixed_output, lstm_hidden_state) 
        # lstm_output = lstm_hidden_state[0] # output is (hidden_state,cell_state), we need hidden state, shape (1,256)
        return lstm_hidden_state

class Agent1(nn.Module):
    def __init__(self, num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions):
        super(Agent1, self).__init__()
        self.language_module = LanguageModule(num_words, embedding_dim)
        self.visual_module = VisualModule()
        self.mixing_module = MixingModule(vision_output_dim, language_output_dim, mixing_dim)
        self.lstm_module = LSTMModule(mixing_dim, lstm_hidden_dim)
        self.action_predictor = nn.Linear(lstm_hidden_dim, num_actions)
        self.value_estimator = nn.Linear(lstm_hidden_dim, 1)

    def forward(self, vt, lt, lstm_hidden_state):
        vision_output = self.visual_module(vt)
        language_output = self.language_module(lt)
        mixed_output = self.mixing_module(vision_output, language_output).unsqueeze(0)
        lstm_output = self.lstm_module(mixed_output,lstm_hidden_state)
        action_probs = self.action_predictor(lstm_output[0]) 
        value_estimate = self.value_estimator(lstm_output[0])
        return action_probs,value_estimate,lstm_output
        
        
    def save(self, episode, ALG_NAME, ENV_ID):
        path = os.path.join('model', '_'.join([ALG_NAME, ENV_ID]))
        if not os.path.exists(path):
            os.makedirs(path)
        torch.save(self.state_dict(), os.path.join(path, f'agent_{episode}.pt'))

    def load(self, episode, ALG_NAME, ENV_ID):
        path = os.path.join('model', '_'.join([ALG_NAME, ENV_ID]))
        self.load_state_dict(torch.load(os.path.join(path, f'agent_{episode}.pt')))    

C:\Users\Palaash.HPZ\anaconda3\envs\transformers_mlagents\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# model w Bert 
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

vision_output_dim = 3136
num_words = 35  # Number of unique words in the vocabulary
bert_output_dim = 768  # output dim of BERT
language_output_dim = 128
embedding_dim = 128
mixing_dim = 256
lstm_hidden_dim = 256
num_actions = 4

# (3,128,128) --> (64,7,7) = 3136 (3-layer CNN)
class VisualModule(nn.Module): 
    def __init__(self):
        super(VisualModule, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=3, padding=0),
            nn.ReLU()
        )
        # self.conv = nn.Sequential(
        #     nn.Conv2d(3, 32, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(32, 64, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2),
        #     nn.ReLU(),
        #     nn.Conv2d(128, 64, kernel_size=5, stride=2, padding=1),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        #     nn.ReLU(),
        #     nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
        #     nn.ReLU(),
        # )

    def forward(self, vt):
        encoded_vt = self.conv(vt)
        return encoded_vt.view(vt.size(0), -1).squeeze()

# one-hot encoding [0 0 1 0 0] --> 128 dimensional embedding (FF)
# S1:5 S2:5 S3:11 S4:9 --> 30 + 5 (noun) = 35 in total
# class LanguageModule(nn.Module): 
#     def __init__(self, num_words, embedding_dim):
#         super(LanguageModule, self).__init__()
#         self.embedding = nn.Linear(num_words, embedding_dim)

#     def forward(self, lt):
#         embedded_lt = self.embedding(lt)
#         return embedded_lt

# bert encoder 
from transformers import BertTokenizer,BertModel

class BertEncoder(nn.Module):
    def __init__(self):
        super(BertEncoder,self).__init__()
        self.bert_tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
        
    def forward(self,input_caption):
        tokens=self.bert_tokenizer.tokenize(input_caption)
        tokens=["[CLS]"]+tokens+["[SEP]"]
#         n_pad_to_add=self.max_words-len(tokens)
#         tokens+=["[PAD]"]*n_pad_to_add
        attention_mask=[1 if token!='[PAD]' else 0 for token in tokens]
        token_ids=self.bert_tokenizer.convert_tokens_to_ids(tokens)
        torch_token_ids=torch.tensor(token_ids).unsqueeze(0)
        torch_attention_mask=torch.tensor(attention_mask).unsqueeze(0)
        return torch_token_ids,torch_attention_mask
        
    
class BertModule(nn.Module):
    def __init__(self,bert_output_dim,embedding_dim):
        super(BertModule,self).__init__()
#         self.max_words=max_words
        self.bert_model=BertModel.from_pretrained("bert-base-uncased")
        self.embedding=nn.Linear(bert_output_dim,embedding_dim)

        # Freeze the weights of the BERT model
        for param in self.bert_model.parameters():
            param.requires_grad = False

    def forward(self,torch_token_ids,torch_attention_mask):
        output=self.bert_model(torch_token_ids,torch_attention_mask)
        output=output[1].view(-1)
        output=self.embedding(output)
        return output
    

# 3136(vision) + 128 (language) --> 256 dimensional embedding (FF)
class MixingModule(nn.Module):
    def __init__(self, vision_output_dim, language_output_dim, mixing_dim):
        super(MixingModule, self).__init__()
        self.linear = nn.Linear(vision_output_dim + language_output_dim, mixing_dim)

    def forward(self, vision_output, language_output):
        combined_output = torch.cat((vision_output, language_output), dim=0)
        mixed_output = self.linear(combined_output)
        return mixed_output

class LSTMModule(nn.Module):
    def __init__(self,mixing_dim,lstm_hidden_dim):
        super(LSTMModule, self).__init__()
        self.lstm = nn.LSTMCell(mixing_dim, lstm_hidden_dim)
    
    def forward(self,mixed_output,lstm_hidden_state):
        lstm_hidden_state = self.lstm(mixed_output, lstm_hidden_state) 
        # lstm_output = lstm_hidden_state[0] # output is (hidden_state,cell_state), we need hidden state, shape (1,256)
        return lstm_hidden_state

class Agent2(nn.Module):
    def __init__(self, bert_output_dim, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions):
        super(Agent2, self).__init__()
        self.bert_language_module = BertModule(bert_output_dim, embedding_dim)
        self.visual_module = VisualModule()
        self.mixing_module = MixingModule(vision_output_dim, language_output_dim, mixing_dim)
        self.lstm_module = LSTMModule(mixing_dim, lstm_hidden_dim)
        self.action_predictor = nn.Linear(lstm_hidden_dim, num_actions)
        self.value_estimator = nn.Linear(lstm_hidden_dim, 1)

    def forward(self, vt, torch_token_ids, torch_attention_mask, lstm_hidden_state):
        vision_output = self.visual_module(vt)
        language_output = self.bert_language_module(torch_token_ids, torch_attention_mask)
        mixed_output = self.mixing_module(vision_output, language_output).unsqueeze(0)
        lstm_output = self.lstm_module(mixed_output,lstm_hidden_state)
        action_probs = self.action_predictor(lstm_output[0]) 
        value_estimate = self.value_estimator(lstm_output[0])
        return action_probs,value_estimate,lstm_output
        
        
    def save(self, episode, ALG_NAME, ENV_ID):
        path = os.path.join('model', '_'.join([ALG_NAME, ENV_ID]))
        if not os.path.exists(path):
            os.makedirs(path)
        torch.save(self.state_dict(), os.path.join(path, f'agent_{episode}.pt'))

    def load(self, episode, ALG_NAME, ENV_ID):
        path = os.path.join('model', '_'.join([ALG_NAME, ENV_ID]))
        self.load_state_dict(torch.load(os.path.join(path, f'agent_{episode}.pt')))    

In [ ]:
#

# import warnings
# # Disable all warnings
# warnings.filterwarnings("ignore")

from transformers import logging
logging.set_verbosity_error()

import argparse
import time
import json
# import matplotlib.pyplot as plt
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
from collections import OrderedDict
# add arguments in command --train/test
# parser = argparse.ArgumentParser(description='Train or test neural net motor controller.')
# parser.add_argument('--train', dest='train', action='store_true', default=False)
# parser.add_argument('--test', dest='test', action='store_true', default=True)
# args = parser.parse_args()
train = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) 

S0_ALG_NAME = 'S2'
S0_ENV_ID = '23'
S0_episode = 24037

ALG_NAME = 'S2'
ENV_ID = '26'

TRAIN_EPISODES = 175267  # number of overall episodes for training
MAX_STEPS = 500  # maximum time step in one episode
LAM = 0.95  # reward discount in TD error
lr = 2.5e-5  #LR
speed = 3
num_steps = 250 # the step for updating the network
max_step_reward = -10

if __name__ == '__main__':
    #initialise pretrained model class 
#     agent = Agent1(num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions)
#     agent.load(S0_episode,S0_ALG_NAME,S0_ENV_ID)
#     pretrained_dict = agent.state_dict()

    # intialise new model class
    agent = Agent2(bert_output_dim, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions)
    agent.load(S0_episode,S0_ALG_NAME,S0_ENV_ID)
#     bert_dict = agent.state_dict()

#     new_dict = OrderedDict()
#     for k in bert_dict.keys():
#         if k in pretrained_dict.keys():
#             new_dict[k] = pretrained_dict[k]
#         else:
#             new_dict[k] = bert_dict[k]
        
#     agent.load_state_dict(new_dict)
    agent.to(device)
    
    optimizer = optim.RMSprop(agent.parameters(), lr=lr)
    best_score = float('-inf')
    hashmap = {
    0: 'capsule',
    1: 'cube',
    2: 'cylinder',
    3: 'prism',
    4: 'sphere',
    5: 'red',
    6: 'green',
    7: 'blue',
    8: 'yellow',
    9: 'black'}
    if train:
        entropy_term = 0
        all_episode_reward = []
        all_average_reward = []
        all_steps = []
        all_actor_loss = []
        all_critic_loss = []
        all_entropy_loss = []
        all_total_loss = []
        tracked_agent = -1
        for episode in range(TRAIN_EPISODES):
            t0 = time.time()
            episode_reward = 0
            # env.reset()
            behavior_name=list(env.behavior_specs)[0]
            spec=env.behavior_specs[behavior_name]
            # state = env.reset().astype(np.float32)
            STEPS = 0
            decision_steps, terminal_steps = env.get_steps(behavior_name)
            # state -- vt, lt, lstm
            vt = torch.tensor(decision_steps.obs[0]).reshape(1,3,128,128).to(device)
            index1 = int(decision_steps.obs[1][0][0])
            index2 = int(decision_steps.obs[1][0][1])+5
            print(f'---{hashmap[index2]} {hashmap[index1]}---')
            input_caption = f"{hashmap[index2]} {hashmap[index1]}"
            bert_encoder=BertEncoder()
            torch_token_ids,torch_attention_mask=bert_encoder(input_caption)
#             text_encoder=BertEncoder(bert_output_dim,num_words)
#             lt=text_encoder(input_caption).to(device).detach()
            torch_token_ids=torch_token_ids.to(device)
            torch_attention_mask=torch_attention_mask.to(device)
        
            # 0-capsule,1-cube,2-cylinder,3-prism,4-sphere 
#             lt = torch.zeros(35).to(device)
#             lt[index1],lt[index2] = 1,1

            lstm_hidden_state = (torch.zeros(1, lstm_hidden_dim).to(device), torch.zeros(1, lstm_hidden_dim).to(device))
            done = False
            while True:

                # Need to use when calculating the loss
                log_probs = []
                # values = []
                values = torch.empty(0).to(device)
                rewards = []

                for steps in range(num_steps):
                    
                    lstm_hidden_state = tuple(tensor.detach() for tensor in lstm_hidden_state)
                    policy_dist, value, lstm_hidden_state = agent(vt,torch_token_ids,torch_attention_mask,lstm_hidden_state)
                    STEPS += 1
                    dist = F.softmax(policy_dist.detach(),dim=1).cpu().numpy()
                    

                    action_dist = Categorical(F.softmax(policy_dist.detach(),dim=1))
                    # action_dist = Categorical(F.softmax(policy_dist,dim=1))
                    action = action_dist.sample() # sample an action from action_dist
                    action_onehot = F.one_hot(torch.tensor(action),num_actions).cpu()
                    
                    log_prob = torch.log(F.softmax(policy_dist,dim=1)[0][action])
                    # log_prob = torch.log(F.softmax(policy_dist,dim=1)[0][action])
                    # entropy = -np.sum(np.mean(dist)* np.log(dist))
                    entropy = F.cross_entropy(policy_dist.detach(), action)

                    discrete_actions = np.array(action_onehot).reshape(1,4)*speed
                    action_tuple = ActionTuple()
                    action_tuple.add_discrete(discrete_actions)
                    env.set_actions(behavior_name,action_tuple)
                    
                    env.step()
                    decision_steps, terminal_steps = env.get_steps(behavior_name)

                    if tracked_agent == -1 and len(decision_steps) >= 1:
                        tracked_agent = decision_steps.agent_id[0]
                        # print(tracked_agent)

                    if tracked_agent in terminal_steps: # roll over or hit the target
                        print('Agent in terminal steps')
                        done = True
                        reward = terminal_steps[tracked_agent].reward
                        if reward > 0:
                            pass
                        else: reward = -1 # roll over or other unseen conditions

                        print(f'Terminal Step reward: {reward}')

                    elif tracked_agent in decision_steps: # the agent which requires action
                        reward = decision_steps[tracked_agent].reward
                        # print(f'Decision Step reward: {reward}')
                        if reward<0:
                            print(f'Decision Step reward: {reward}')
                            # if reward<-1: hit = 1
                    if STEPS >= MAX_STEPS:
                        reward = max_step_reward
                        print(f'Max Step Reward: {reward}')
                        env.reset()
                        done = True
                    if STEPS % num_steps == 0:
                        print (f'Step: {STEPS}')

                    episode_reward = episode_reward + reward

                    rewards.append(reward)
                    # values.append(value)
                    values = torch.cat((values, value), dim=0)
                    log_probs.append(log_prob)
                    entropy_term = entropy_term + entropy
                    vt_new = torch.tensor(decision_steps.obs[0]).reshape(1,3,128,128).to(device)
                    vt = vt_new

                    if done or steps == num_steps-1:
                        # _, Qval,_ = agent(vt_new,lt,lstm_hidden_state)
                        # Qval = Qval.detach()
                        break
                
                
                discounted_rewards = np.zeros_like(values.cpu().detach().numpy())
                cumulative = 0
                # print(len(rewards))
                for t in reversed(range(len(rewards))):
                    cumulative = rewards[t] + LAM * cumulative # Monte Carlo
                    discounted_rewards[t] = cumulative
                # print(f'rewards:{rewards}, discounted_rewards:{discounted_rewards}')
                # Advantage Actor Critic

                # Qvals[-1] = rewards[t] + LAM * Qval      or       Qvals[-1] = rewards[t]                   
                # for t in range(len(rewards)-1):
                #         Qvals[t] = rewards[t] + LAM * values[t+1]
                
                # r_(t+1) = R(s_t|a_t)--> reward[t]        a_t, V_t = agent(s_t)
                # A_t = r_(t+1) + LAM * V_(t+1) - V_t 
                #     = Q_t - V_t
                
                # Monte Carlo Advantage = reward + LAM * cumulative_reward
                # Actor_loss = -log(pai(s_t|a_t))*A_t
                # Critic_loss = A_t.pow(2) *0.5
                # Entropy_loss = -F.entropy(pai(St),index) * 0.001

                # entropy = -np.sum(np.mean(dist) * np.log(dist))
                
                #update actor critic
                
                # values = torch.FloatTensor(values).requires_grad_(True).to(device)
                discounted_rewards = torch.FloatTensor(discounted_rewards.astype(np.float32)).to(device)
                log_probs = torch.stack(log_probs)
                advantage = discounted_rewards - values
                actor_loss = (-log_probs * advantage).mean()
                critic_loss = 0.5 * torch.square(advantage).mean()
                entropy_term /= num_steps
                entropy_loss = -0.1 * entropy_term
                ac_loss = actor_loss + critic_loss + entropy_loss
                # ac_loss = values.mean()
                optimizer.zero_grad()
                ac_loss.backward()
                optimizer.step()
                # print('updated')
                # for name, param in agent.named_parameters():
                #     if param.grad is not None:
                #         print(name, param.grad)
                #     else:
                #         print(name, "gradients not computed")
                # for name, param in agent.named_parameters():
                #     if name == 'value_estimator.weight':
                #         print(name, param)
                
                
                if done: break


            all_episode_reward.append(float(episode_reward))
            all_steps.append(STEPS)
            all_actor_loss.append(float(actor_loss))
            all_critic_loss.append(float(critic_loss))
            all_entropy_loss.append(float(entropy_loss))
            all_total_loss.append(float(ac_loss))
            if episode >= 100:
                avg_score = np.mean(all_episode_reward[-100:])
                all_average_reward.append(avg_score)
                if avg_score > best_score:
                    best_score = avg_score
                    agent.save(episode, ALG_NAME, ENV_ID)
                    print(f'-----The best score for averaging previous 200 episode reward is {best_score}. Model has been saved-----')
                print('Training  | Episode: {}/{}  | Episode Reward: {:.1f}  | Average Reward {:.2f}  | Actor loss: {:.2f} | Critic loss: {:.2f} | Entropy loss: {:.4f}  | Total Loss: {:.2f} | Total Steps: {}' \
                    .format(episode + 1, TRAIN_EPISODES, episode_reward, avg_score, actor_loss, critic_loss,entropy_loss,  ac_loss, STEPS))
            else:  print('Training  | Episode: {}/{}  | Episode Reward: {:.1f}  | Actor loss: {:.2f} | Critic loss: {:.2f} | Entropy loss: {:.4f}  | Total Loss: {:.2f} | Total Steps: {}' \
                    .format(episode + 1, TRAIN_EPISODES, episode_reward, actor_loss, critic_loss, entropy_loss,  ac_loss, STEPS))
            if episode%5000 == 0:
                    agent.save(episode, ALG_NAME, ENV_ID)
                    print("Model has been saved")
        print(all_average_reward)
        agent.save(episode ,ALG_NAME, ENV_ID)
        print("Model has been saved")

        data = {
                    'all_average_reward': all_average_reward,
                    'all_episode_reward': all_episode_reward,
                    'all_actor_loss': all_actor_loss,
                    'all_critic_loss': all_critic_loss,
                    'all_entropy_loss': all_entropy_loss,
                    'all_total_loss': all_total_loss,
                    'all_steps': all_steps,
                } 
        file_path = f'result/{ALG_NAME}_{ENV_ID}.txt'
        with open(file_path, 'w') as file:
            json.dump(data, file)



cuda
---black capsule---


C:\Users\Palaash.HPZ\anaconda3\envs\transformers_mlagents\lib\site-packages\ipykernel_launcher.py:136: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Decision Step reward: -2.5
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1/175267  | Episode Reward: 5.0  | Actor loss: -0.15 | Critic loss: 8.13 | Entropy loss: -0.0032  | Total Loss: 7.98 | Total Steps: 44
Model has been saved
---green prism---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 2/175267  | Episode Reward: 10.0  | Actor loss: 0.02 | Critic loss: 6.68 | Entropy loss: -0.0000  | Total Loss: 6.69 | Total Steps: 6
---blue sphere---
Decision Step reward: -2.5
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 3/175267  | Episode Reward: 5.0  | Actor loss: 0.05 | Critic loss: 6.50 | Entropy loss: -0.0027  | Total Loss: 6.55 | Total Steps: 56
---yellow cylinder---
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 4/175267  | Episode Reward: 7.5  | Actor loss: 0.30 | Critic loss: 4.33 | Entropy loss: -0.0025

Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 33/175267  | Episode Reward: 10.0  | Actor loss: 0.01 | Critic loss: 1.27 | Entropy loss: -0.0000  | Total Loss: 1.27 | Total Steps: 6
---yellow capsule---
Decision Step reward: -2.5
Decision Step reward: -1.0
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 34/175267  | Episode Reward: 6.5  | Actor loss: -0.35 | Critic loss: 5.43 | Entropy loss: -0.0027  | Total Loss: 5.08 | Total Steps: 45
---green sphere---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 35/175267  | Episode Reward: 10.0  | Actor loss: -0.25 | Critic loss: 1.09 | Entropy loss: -0.0020  | Total Loss: 0.84 | Total Steps: 7
---black cylinder---
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 36/175267  | Episode Reward: 7.5  | Actor loss: -0.23 | Critic loss: 2.49 | Entropy loss: -0.0042  | Total Loss: 2.26 | Total Steps: 75
---yellow cube---
Decision 

Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 65/175267  | Episode Reward: 7.5  | Actor loss: 0.17 | Critic loss: 7.41 | Entropy loss: -0.0006  | Total Loss: 7.58 | Total Steps: 30
---black cube---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 66/175267  | Episode Reward: 10.0  | Actor loss: 0.11 | Critic loss: 2.96 | Entropy loss: -0.0005  | Total Loss: 3.07 | Total Steps: 30
---green cylinder---
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 67/175267  | Episode Reward: 7.5  | Actor loss: 0.07 | Critic loss: 3.99 | Entropy loss: -0.0004  | Total Loss: 4.06 | Total Steps: 34
---black prism---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 68/175267  | Episode Reward: 10.0  | Actor loss: 0.01 | Critic loss: 0.86 | Entropy loss: -0.0000  | Total Loss: 0.87 | Total Steps: 6
---blue cube---
Decision Step reward: -2.5
Decision Step reward: -2.5
Decision Step rewa

...

Decision Step reward: -2.5
Decision Step reward: -2.5
Decision Step reward: -2.5
Decision Step reward: -2.5
Step: 250
Decision Step reward: -2.5
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1555/175267  | Episode Reward: -7.5  | Average Reward 6.83  | Actor loss: -0.33 | Critic loss: 5.71 | Entropy loss: -0.0045  | Total Loss: 5.37 | Total Steps: 304
---blue capsule---
Decision Step reward: -2.5
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1556/175267  | Episode Reward: 5.0  | Average Reward 6.80  | Actor loss: -0.19 | Critic loss: 5.68 | Entropy loss: -0.0055  | Total Loss: 5.48 | Total Steps: 121
---red prism---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1557/175267  | Episode Reward: 10.0  | Average Reward 6.83  | Actor loss: 0.01 | Critic loss: 1.37 | Entropy loss: -0.0000  | Total Loss: 1.38 | Total Steps: 6
---green sphere---
Decision Step rewar

Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1584/175267  | Episode Reward: 5.0  | Average Reward 6.88  | Actor loss: 0.14 | Critic loss: 6.75 | Entropy loss: -0.0018  | Total Loss: 6.88 | Total Steps: 53
---yellow capsule---
Decision Step reward: -2.5
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1585/175267  | Episode Reward: 5.0  | Average Reward 6.85  | Actor loss: -0.32 | Critic loss: 8.03 | Entropy loss: -0.0025  | Total Loss: 7.70 | Total Steps: 73
---red cube---
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1586/175267  | Episode Reward: 7.5  | Average Reward 6.85  | Actor loss: 0.06 | Critic loss: 7.23 | Entropy loss: -0.0006  | Total Loss: 7.30 | Total Steps: 39
---blue sphere---
Decision Step reward: -2.5
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1587/175267  | 

Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1613/175267  | Episode Reward: 10.0  | Average Reward 6.84  | Actor loss: 0.01 | Critic loss: 0.21 | Entropy loss: -0.0000  | Total Loss: 0.22 | Total Steps: 6
---black cube---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1614/175267  | Episode Reward: 10.0  | Average Reward 6.84  | Actor loss: 0.03 | Critic loss: 4.34 | Entropy loss: -0.0002  | Total Loss: 4.37 | Total Steps: 31
---green prism---
Decision Step reward: -2.5
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1615/175267  | Episode Reward: 5.0  | Average Reward 6.84  | Actor loss: -0.27 | Critic loss: 5.89 | Entropy loss: -0.0024  | Total Loss: 5.62 | Total Steps: 49
---green prism---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1616/175267  | Episode Reward: 10.0  | Average Reward 6.87  | Actor loss: 0.01 | Critic loss: 2.10 | Entropy loss: -0.0000 

Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1643/175267  | Episode Reward: 10.0  | Average Reward 7.21  | Actor loss: 0.61 | Critic loss: 2.73 | Entropy loss: -0.0042  | Total Loss: 3.34 | Total Steps: 32
---green cylinder---
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1644/175267  | Episode Reward: 7.5  | Average Reward 7.18  | Actor loss: -0.00 | Critic loss: 0.98 | Entropy loss: -0.0006  | Total Loss: 0.98 | Total Steps: 38
---yellow sphere---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1645/175267  | Episode Reward: 10.0  | Average Reward 7.18  | Actor loss: 0.01 | Critic loss: 0.36 | Entropy loss: -0.0000  | Total Loss: 0.37 | Total Steps: 6
---black capsule---
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1646/175267  | Episode Reward: 7.5  | Average Reward 7.18  | Actor loss: 0.31 | Critic loss: 4.69 | Entropy loss: -

Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1672/175267  | Episode Reward: 10.0  | Average Reward 7.16  | Actor loss: 0.07 | Critic loss: 3.18 | Entropy loss: -0.0008  | Total Loss: 3.25 | Total Steps: 30
---yellow cylinder---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1673/175267  | Episode Reward: 10.0  | Average Reward 7.21  | Actor loss: 0.04 | Critic loss: 2.86 | Entropy loss: -0.0010  | Total Loss: 2.90 | Total Steps: 43
---green capsule---
Decision Step reward: -2.5
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1674/175267  | Episode Reward: 7.5  | Average Reward 7.25  | Actor loss: 0.06 | Critic loss: 7.86 | Entropy loss: -0.0004  | Total Loss: 7.92 | Total Steps: 32
---green capsule---
Agent in terminal steps
Terminal Step reward: 10.0
Training  | Episode: 1675/175267  | Episode Reward: 10.0  | Average Reward 7.28  | Actor loss: 0.01 | Critic loss: 1.59 | Entropy loss: -0.0000  | Total Loss: 1.6

In [9]:

        data = {
                    'all_average_reward': all_average_reward,
                    'all_episode_reward': all_episode_reward,
                    'all_actor_loss': all_actor_loss,
                    'all_critic_loss': all_critic_loss,
                    'all_entropy_loss': all_entropy_loss,
                    'all_total_loss': all_total_loss,
                    'all_steps': all_steps,
                } 
        file_path = f'result/{ALG_NAME}_{ENV_ID}.txt'
        with open(file_path, 'w') as file:
            json.dump(data, file)


# training loop to certain parts of pretrained model

In [10]:
#

# import warnings
# # Disable all warnings
# warnings.filterwarnings("ignore")

from transformers import logging
logging.set_verbosity_error()

import argparse
import time
import json
# import matplotlib.pyplot as plt
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
from collections import OrderedDict
# add arguments in command --train/test
# parser = argparse.ArgumentParser(description='Train or test neural net motor controller.')
# parser.add_argument('--train', dest='train', action='store_true', default=False)
# parser.add_argument('--test', dest='test', action='store_true', default=True)
# args = parser.parse_args()
train = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) 

S0_ALG_NAME = 'S2'
S0_ENV_ID = '23'
S0_episode = 24037

ALG_NAME = 'S2'
ENV_ID = '26'

TRAIN_EPISODES = 175267  # number of overall episodes for training
MAX_STEPS = 500  # maximum time step in one episode
LAM = 0.95  # reward discount in TD error
lr = 2.5e-5  #LR
speed = 3
num_steps = 250 # the step for updating the network
max_step_reward = -10

if __name__ == '__main__':
    #initialise pretrained model class 
    agent = Agent1(num_words, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions)
    agent.load(S0_episode,S0_ALG_NAME,S0_ENV_ID)
    pretrained_dict = agent.state_dict()

    # intialise new model class
    agent = Agent2(bert_output_dim, embedding_dim, vision_output_dim, language_output_dim, mixing_dim, lstm_hidden_dim,num_actions)
    bert_dict = agent.state_dict()

    new_dict = OrderedDict()
    for k in bert_dict.keys():
        if k in pretrained_dict.keys():
            new_dict[k] = pretrained_dict[k]
        else:
            new_dict[k] = bert_dict[k]
        
    agent.load_state_dict(new_dict)
    agent.to(device)
    
    optimizer = optim.RMSprop(agent.parameters(), lr=lr)
    best_score = float('-inf')
    hashmap = {
    0: 'capsule',
    1: 'cube',
    2: 'cylinder',
    3: 'prism',
    4: 'sphere',
    5: 'red',
    6: 'green',
    7: 'blue',
    8: 'yellow',
    9: 'black'}
    if train:
        entropy_term = 0
        all_episode_reward = []
        all_average_reward = []
        all_steps = []
        all_actor_loss = []
        all_critic_loss = []
        all_entropy_loss = []
        all_total_loss = []
        tracked_agent = -1
        for episode in range(TRAIN_EPISODES):
            t0 = time.time()
            episode_reward = 0
            # env.reset()
            behavior_name=list(env.behavior_specs)[0]
            spec=env.behavior_specs[behavior_name]
            # state = env.reset().astype(np.float32)
            STEPS = 0
            decision_steps, terminal_steps = env.get_steps(behavior_name)
            # state -- vt, lt, lstm
            vt = torch.tensor(decision_steps.obs[0]).reshape(1,3,128,128).to(device)
            index1 = int(decision_steps.obs[1][0][0])
            index2 = int(decision_steps.obs[1][0][1])+5
            print(f'---{hashmap[index2]} {hashmap[index1]}---')
            input_caption = f"{hashmap[index2]} {hashmap[index1]}"
            bert_encoder=BertEncoder()
            torch_token_ids,torch_attention_mask=bert_encoder(input_caption)
#             text_encoder=BertEncoder(bert_output_dim,num_words)
#             lt=text_encoder(input_caption).to(device).detach()
            torch_token_ids=torch_token_ids.to(device)
            torch_attention_mask=torch_attention_mask.to(device)
        
            # 0-capsule,1-cube,2-cylinder,3-prism,4-sphere 
#             lt = torch.zeros(35).to(device)
#             lt[index1],lt[index2] = 1,1

            lstm_hidden_state = (torch.zeros(1, lstm_hidden_dim).to(device), torch.zeros(1, lstm_hidden_dim).to(device))
            done = False
            while True:

                # Need to use when calculating the loss
                log_probs = []
                # values = []
                values = torch.empty(0).to(device)
                rewards = []

                for steps in range(num_steps):
                    
                    lstm_hidden_state = tuple(tensor.detach() for tensor in lstm_hidden_state)
                    policy_dist, value, lstm_hidden_state = agent(vt,torch_token_ids,torch_attention_mask,lstm_hidden_state)
                    STEPS += 1
                    dist = F.softmax(policy_dist.detach(),dim=1).cpu().numpy()
                    

                    action_dist = Categorical(F.softmax(policy_dist.detach(),dim=1))
                    # action_dist = Categorical(F.softmax(policy_dist,dim=1))
                    action = action_dist.sample() # sample an action from action_dist
                    action_onehot = F.one_hot(torch.tensor(action),num_actions).cpu()
                    
                    log_prob = torch.log(F.softmax(policy_dist,dim=1)[0][action])
                    # log_prob = torch.log(F.softmax(policy_dist,dim=1)[0][action])
                    # entropy = -np.sum(np.mean(dist)* np.log(dist))
                    entropy = F.cross_entropy(policy_dist.detach(), action)

                    discrete_actions = np.array(action_onehot).reshape(1,4)*speed
                    action_tuple = ActionTuple()
                    action_tuple.add_discrete(discrete_actions)
                    env.set_actions(behavior_name,action_tuple)
                    
                    env.step()
                    decision_steps, terminal_steps = env.get_steps(behavior_name)

                    if tracked_agent == -1 and len(decision_steps) >= 1:
                        tracked_agent = decision_steps.agent_id[0]
                        # print(tracked_agent)

                    if tracked_agent in terminal_steps: # roll over or hit the target
                        print('Agent in terminal steps')
                        done = True
                        reward = terminal_steps[tracked_agent].reward
                        if reward > 0:
                            pass
                        else: reward = -1 # roll over or other unseen conditions

                        print(f'Terminal Step reward: {reward}')

                    elif tracked_agent in decision_steps: # the agent which requires action
                        reward = decision_steps[tracked_agent].reward
                        # print(f'Decision Step reward: {reward}')
                        if reward<0:
                            print(f'Decision Step reward: {reward}')
                            # if reward<-1: hit = 1
                    if STEPS >= MAX_STEPS:
                        reward = max_step_reward
                        print(f'Max Step Reward: {reward}')
                        env.reset()
                        done = True
                    if STEPS % num_steps == 0:
                        print (f'Step: {STEPS}')

                    episode_reward = episode_reward + reward

                    rewards.append(reward)
                    # values.append(value)
                    values = torch.cat((values, value), dim=0)
                    log_probs.append(log_prob)
                    entropy_term = entropy_term + entropy
                    vt_new = torch.tensor(decision_steps.obs[0]).reshape(1,3,128,128).to(device)
                    vt = vt_new

                    if done or steps == num_steps-1:
                        # _, Qval,_ = agent(vt_new,lt,lstm_hidden_state)
                        # Qval = Qval.detach()
                        break
                
                
                discounted_rewards = np.zeros_like(values.cpu().detach().numpy())
                cumulative = 0
                # print(len(rewards))
                for t in reversed(range(len(rewards))):
                    cumulative = rewards[t] + LAM * cumulative # Monte Carlo
                    discounted_rewards[t] = cumulative
                # print(f'rewards:{rewards}, discounted_rewards:{discounted_rewards}')
                # Advantage Actor Critic

                # Qvals[-1] = rewards[t] + LAM * Qval      or       Qvals[-1] = rewards[t]                   
                # for t in range(len(rewards)-1):
                #         Qvals[t] = rewards[t] + LAM * values[t+1]
                
                # r_(t+1) = R(s_t|a_t)--> reward[t]        a_t, V_t = agent(s_t)
                # A_t = r_(t+1) + LAM * V_(t+1) - V_t 
                #     = Q_t - V_t
                
                # Monte Carlo Advantage = reward + LAM * cumulative_reward
                # Actor_loss = -log(pai(s_t|a_t))*A_t
                # Critic_loss = A_t.pow(2) *0.5
                # Entropy_loss = -F.entropy(pai(St),index) * 0.001

                # entropy = -np.sum(np.mean(dist) * np.log(dist))
                
                #update actor critic
                
                # values = torch.FloatTensor(values).requires_grad_(True).to(device)
                discounted_rewards = torch.FloatTensor(discounted_rewards.astype(np.float32)).to(device)
                log_probs = torch.stack(log_probs)
                advantage = discounted_rewards - values
                actor_loss = (-log_probs * advantage).mean()
                critic_loss = 0.5 * torch.square(advantage).mean()
                entropy_term /= num_steps
                entropy_loss = -0.1 * entropy_term
                ac_loss = actor_loss + critic_loss + entropy_loss
                # ac_loss = values.mean()
                optimizer.zero_grad()
                ac_loss.backward()
                optimizer.step()
                # print('updated')
                # for name, param in agent.named_parameters():
                #     if param.grad is not None:
                #         print(name, param.grad)
                #     else:
                #         print(name, "gradients not computed")
                # for name, param in agent.named_parameters():
                #     if name == 'value_estimator.weight':
                #         print(name, param)
                
                
                if done: break


            all_episode_reward.append(float(episode_reward))
            all_steps.append(STEPS)
            all_actor_loss.append(float(actor_loss))
            all_critic_loss.append(float(critic_loss))
            all_entropy_loss.append(float(entropy_loss))
            all_total_loss.append(float(ac_loss))
            if episode >= 100:
                avg_score = np.mean(all_episode_reward[-100:])
                all_average_reward.append(avg_score)
                if avg_score > best_score:
                    best_score = avg_score
                    agent.save(episode, ALG_NAME, ENV_ID)
                    print(f'-----The best score for averaging previous 200 episode reward is {best_score}. Model has been saved-----')
                print('Training  | Episode: {}/{}  | Episode Reward: {:.1f}  | Average Reward {:.2f}  | Actor loss: {:.2f} | Critic loss: {:.2f} | Entropy loss: {:.4f}  | Total Loss: {:.2f} | Total Steps: {}' \
                    .format(episode + 1, TRAIN_EPISODES, episode_reward, avg_score, actor_loss, critic_loss,entropy_loss,  ac_loss, STEPS))
            else:  print('Training  | Episode: {}/{}  | Episode Reward: {:.1f}  | Actor loss: {:.2f} | Critic loss: {:.2f} | Entropy loss: {:.4f}  | Total Loss: {:.2f} | Total Steps: {}' \
                    .format(episode + 1, TRAIN_EPISODES, episode_reward, actor_loss, critic_loss, entropy_loss,  ac_loss, STEPS))
            if episode%5000 == 0:
                    agent.save(episode, ALG_NAME, ENV_ID)
                    print("Model has been saved")
        print(all_average_reward)
        agent.save(episode ,ALG_NAME, ENV_ID)
        print("Model has been saved")

        data = {
                    'all_average_reward': all_average_reward,
                    'all_episode_reward': all_episode_reward,
                    'all_actor_loss': all_actor_loss,
                    'all_critic_loss': all_critic_loss,
                    'all_entropy_loss': all_entropy_loss,
                    'all_total_loss': all_total_loss,
                    'all_steps': all_steps,
                } 
        file_path = f'result/{ALG_NAME}_{ENV_ID}.txt'
        with open(file_path, 'w') as file:
            json.dump(data, file)



cuda


RuntimeError: Error(s) in loading state_dict for Agent1:
	Missing key(s) in state_dict: "language_module.embedding.weight", "language_module.embedding.bias". 
	Unexpected key(s) in state_dict: "bert_language_module.bert_model.embeddings.position_ids", "bert_language_module.bert_model.embeddings.word_embeddings.weight", "bert_language_module.bert_model.embeddings.position_embeddings.weight", "bert_language_module.bert_model.embeddings.token_type_embeddings.weight", "bert_language_module.bert_model.embeddings.LayerNorm.weight", "bert_language_module.bert_model.embeddings.LayerNorm.bias", "bert_language_module.bert_model.encoder.layer.0.attention.self.query.weight", "bert_language_module.bert_model.encoder.layer.0.attention.self.query.bias", "bert_language_module.bert_model.encoder.layer.0.attention.self.key.weight", "bert_language_module.bert_model.encoder.layer.0.attention.self.key.bias", "bert_language_module.bert_model.encoder.layer.0.attention.self.value.weight", "bert_language_modul